<font color='blue'> Привет! Поздравляю с первым самостоятельным проектом. Вижу, ты хорошо поработал. Ниже мои комментарии выделены синим..</font>

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd

Прочитаем файл datasets/data.csv и сохраним его в переменной solvency.

In [2]:
solvency = pd.read_csv('/datasets/data.csv')

Запросим первые 5 строк таблицы для оценки её структуры.

In [3]:
solvency.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Запросим общую информацию о таблице для оценки объёма данных.

In [4]:
solvency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Просмотрим уникальные значения по столбцам, в которых это целесообразно, и попробуем выявить проблемы в данных:

In [5]:
solvency['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [6]:
solvency['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [7]:
solvency['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [8]:
solvency['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [9]:
solvency['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [10]:
solvency['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

### Вывод

Таблица состоит из 12 столбцов.

Тип данных "целые числа" в столбцах:
- children;
- dob_years;
- education_id;        
- family_status_id;
- debt.

Тип данных "вещественные числа" в столбцах:
- days_employed;
- total_income.  

Тип данных "object" в столбцах:
- education;           
- family_status;
- gender;
- income_type;
- purpose.

Столбцы содержат следующую информацию:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита


Первый взгляд на данные позволяет увидеть следующие проблемы:
- отрицательные значения в столбце children;
- отрицательные значения в столбце days_employed;
- несовпадение регистров записей в столбце education;
- нулевые значение в столбце dob_years;
- XNA в столбце gender;
- пропуски в столбце days_employed;
- пропуски в столбце total_income.

<font color='blue'> Хорошая и очень тщательная предобработка сырых данных.</font>

### Шаг 2. Предобработка данных

### Обработка пропусков

Оцениваем наличие пропусков в ячейках.

In [11]:
solvency.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски содержаться в столбцах days_employed и total_income. Необходимо сообщить коллегам о данной проблеме.

Чтобы продолжить работу, попытаемся самостоятельно решить проблему. Избавимся от пропусков в столбце total_income. Заменим "NaN" на медианные значения, при этом обратим внимание, что для разных групп занятости требуются свои медианные значения.

In [12]:
solvency['total_income'] = solvency.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.mean()))

<font color='blue'> Это возможный вариант заполнить пропуски. Однако в доходах есть достаточно силдьный разброс и выбросы, это значит, лучше заполнять медианой.</font>

В столбце days_employed, кроме пропусков значений, на этапе оценки структуры данных мы выявили и другую проблему - отрицательные значения. Предполагаем, что могут существовать и другие "подводные камни".

Так как мы хотим не просто получить результат, а достичь его максимально быстро и эффективно, принимаем решение уклонения от потенциального риска - удалим стобец days_employed, так как он не требуtтся для решения поставленной задачи.

In [13]:
del solvency ['days_employed']

<font color='blue'> Возможный вариант.</font>

Проверим результат:

In [14]:
solvency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 1.8+ MB


<font color='blue'> Отлично, пропуски заполнены - можно работать дальше.</font>

Разберемся с отрицательными значениями в столбце children:

In [15]:
solvency[solvency['children'] == -1]

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование
705,-1,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля
742,-1,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование
800,-1,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293.724153,дополнительное образование
941,-1,57,Среднее,1,женат / замужем,0,F,пенсионер,0,137127.465690,на покупку своего автомобиля
1363,-1,55,СРЕДНЕЕ,1,женат / замужем,0,F,компаньон,0,69550.699692,профильное образование
1929,-1,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121.569013,покупка жилья
2073,-1,42,среднее,1,в разводе,3,F,компаньон,0,162638.609373,покупка жилья
3814,-1,26,Среднее,1,гражданский брак,1,F,госслужащий,0,131892.785435,на проведение свадьбы
4201,-1,41,среднее,1,женат / замужем,0,F,госслужащий,0,226375.766751,операции со своей недвижимостью


In [16]:
solvency[solvency['children'] == -1]['children'].count()

47

Возможны два варианта корректировки данных:
1. Взять значения по модулю
2. Очистить некорректные данные

Так как мы не знаем в чем именно заключается ошибка заполнения данных, а также принимаем, что ~0.2% данных от общей выборки не является значительным для анализа объемом, приходим к решению избавиться от некорректных данных.
Отфильтруем датафрейм по условию полжительных значений в столбце children: 

In [17]:
solvency = solvency[solvency['children'] >= 0]

<font color='blue'> Хорошо, правильный вариант.</font>

Проверим результат:

In [18]:
solvency.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21478 entries, 0 to 21524
Data columns (total 11 columns):
children            21478 non-null int64
dob_years           21478 non-null int64
education           21478 non-null object
education_id        21478 non-null int64
family_status       21478 non-null object
family_status_id    21478 non-null int64
gender              21478 non-null object
income_type         21478 non-null object
debt                21478 non-null int64
total_income        21478 non-null float64
purpose             21478 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 2.0+ MB


Разберемся с нулевыми значениями в dob_years:

In [19]:
solvency[solvency['dob_years'] == 0]

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...
19829,0,0,среднее,1,женат / замужем,0,F,сотрудник,0,161380.260488,жилье
20462,0,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья
20577,0,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость
21179,2,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости


Заменим нулевые значения типичными значениями столбца:

In [20]:
dob_years_mean = solvency['dob_years'].mean()
solvency['dob_years'] = solvency['dob_years'].replace(0.00, dob_years_mean)

<font color='blue'> Хорошо.</font>

Проверим результат:

In [21]:
solvency[solvency['dob_years'] == 0]

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Разберемся с XNA в столбце gender:

In [22]:
solvency[solvency['gender'] == 'XNA']

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,24.0,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Удалим строку с некорректными данными:

In [23]:
solvency = solvency[solvency['gender'] != 'XNA']

Проверим результат:

In [24]:
solvency[solvency['gender'] == 'XNA']

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [25]:
solvency.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21477 entries, 0 to 21524
Data columns (total 11 columns):
children            21477 non-null int64
dob_years           21477 non-null float64
education           21477 non-null object
education_id        21477 non-null int64
family_status       21477 non-null object
family_status_id    21477 non-null int64
gender              21477 non-null object
income_type         21477 non-null object
debt                21477 non-null int64
total_income        21477 non-null float64
purpose             21477 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 2.0+ MB


Ранее при изучении данных мы обнаружили, что записи в столбце education сделаны в разных регистрах.
В будущем это может помешать нам найти все дубликаты. Приведем все записи в столбце к единообразному виду:

In [26]:
solvency['education'] = solvency['education'].str.lower()

<font color='blue'> Отлично, важную вешь заметили.</font>

Проверим результат:

In [27]:
print(solvency['education'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


### Вывод

В ходе 1-й части предобработки данных разобрались со всеми проблемами, озвученными при изучении данных:
- отрицательные значения в столбце children;
- отрицательные значения в столбце days_employed;
- несовпадение регистров записей в столбце education;
- нулевые значение в столбце dob_years;
- XNA в столбце gender;
- пропуски в столбце days_employed;
- пропуски в столбце total_income.

 Можем двигаться дальше.

<font color='blue'> Да, можно, отличное заполнение пропусков явных и неявных.</font>

### Замена типа данных

В процессе корректировки значений столбца dob_years тип данных превратился в вещественный.
Вернем его к правильной форме. Наиболее простой способ сделать это - присвоить столбцу тип integer с помощью метода astype:

In [28]:
solvency['dob_years'] = solvency['dob_years'].astype(int)

Проверим результат:

In [29]:
solvency.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21477 entries, 0 to 21524
Data columns (total 11 columns):
children            21477 non-null int64
dob_years           21477 non-null int64
education           21477 non-null object
education_id        21477 non-null int64
family_status       21477 non-null object
family_status_id    21477 non-null int64
gender              21477 non-null object
income_type         21477 non-null object
debt                21477 non-null int64
total_income        21477 non-null float64
purpose             21477 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 2.0+ MB


<font color='blue'> Еще можно total_income сделать int - нам вряд ли нужны копейки.</font>

### Вывод

Теперь все значения в столбцах соответствуют подходящему типу данных. Решение удалить days_employed позволило не разбираться с float в этом столбце. Можем двигаться дальше.

### Обработка дубликатов

Так как информация в столбцах содержит категориальное описание заёмщиков, использовать метод value_count для выявления дубликатов не имеет смысла. Однако, если у некоторых строк значения по всем столбцам будут повторяться, можем предположить, что это действительно дублирирование данных.

Оценим количество дублированных строк:

In [30]:
solvency.duplicated().sum()

71

В данных 71 идентичная строка.

Можно предположить, что у некоторых заёмщиков действительно совпадают данные по выбранным критериям.

Для повышения точности обнаружения дубликатов, закомментируем строку удаления столбца days_employed. Данным в столбце days_employed присвоен тип float64 (т.е. досаточно точные значения за счет знаков после запятой),если строки будут совпадать и по этому дополнительному критерию, можем быть уверены - идентичные строки действительно являются технической ошибкой.

После подсчета дубликатов вместе со значениями столбца days_employed количество идентичных строк не изменилось. Гипотеза подтвердилась - дубликаты возникли по причине технчиеской ошибки и от них стоит избавиться, а также нужно сообщить об обнаруженной проблеме коллегам.

In [31]:
solvency = solvency.drop_duplicates()

Проверим результат:

In [32]:
solvency.duplicated().sum()

0

In [33]:
solvency.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21406 entries, 0 to 21524
Data columns (total 11 columns):
children            21406 non-null int64
dob_years           21406 non-null int64
education           21406 non-null object
education_id        21406 non-null int64
family_status       21406 non-null object
family_status_id    21406 non-null int64
gender              21406 non-null object
income_type         21406 non-null object
debt                21406 non-null int64
total_income        21406 non-null float64
purpose             21406 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 2.0+ MB


<font color='blue'> Отлично, дубликаты удалили - это важный шаг.</font>

### Вывод

Входные данные не очень богаты на дубликаты - 71 дублированная строка составлет ~0.3% от общей выборки. Без зазрения совести оставляем в датафрейме 21 406 строк и двигаемся дальше.

### Лемматизация

Посмотрим на уникальные значения столбца purpose:

In [34]:
solvency['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

"Глазами" можем выделить следующие группы слов:
- жилье;
- недвижимость;
- кваритра;
- свадьба;
- автомобиль;
- образование.

Добавим библеотеку Mystem и напишем функцию для лемматизации:

In [35]:
from pymystem3 import Mystem
m = Mystem()

In [36]:
def lemmatizator(i):
    lemmas = m.lemmatize(i)
    return lemmas

Добавим в датафрейм столбец, в котором будут содержаться леммы:

In [37]:
solvency['purpose_lemmas'] = solvency['purpose'].apply(lemmatizator)

Напишем функцию для определения лемм:

In [38]:
def purpose_type(purpose_lemmas):
    if 'жилье' in purpose_lemmas:
        return 'недвижимость'
    elif 'недвижимость' in purpose_lemmas:
        return 'недвижимость'
    elif 'квартира' in purpose_lemmas:
        return 'недвижимость'
    elif 'свадьба' in purpose_lemmas:
        return 'свадьба'
    elif 'автомобиль' in purpose_lemmas:
        return 'автомобиль'
    elif 'образование' in purpose_lemmas:
        return 'образование'
    else:
        return 'без категории'


Добавим в датафрейм столбец, в котором будут содержаться группы лемм:

In [39]:
solvency['purpose_type'] = solvency['purpose_lemmas'].apply(purpose_type)
solvency['purpose_type'].value_counts()

недвижимость    10786
автомобиль       4295
образование      4003
свадьба          2322
Name: purpose_type, dtype: int64

### Вывод

Удалось выделить основные группы потребностей в  займах - недвижимость, автомобиль, образование, свадьба. 

Категория "недвижимость" с отрывом лидирует среди остальных групп. Это выглядит логично - миллениалы, отказывающиеся от покупки недвижимости, как фактора ограничения свободы, и не являются целевой аудиторий кредитных организаций. Для остальных покупка недвижимости остается важной целью, требующей при этом серьезных капиталовложений - кредитование в помощь.

<font color='blue'> лемматизация выполнена правильно - это самая сложная часть проекта. Заодно и категоризацию начали, ее можно было перенести на следующий шаг.</font>

### Категоризация данных

Для ответа на основные вопросы исследования нам не хватает категоризации заемщиков по количеству детей и уровню дохода.

Напишем функцию для категоризации по количеству детей:

In [40]:
def children_group(amount_children):
    if amount_children == 0:
        return 'без детей'
    if (amount_children > 0) & (amount_children <= 1):
        return 'один ребенок'
    if (amount_children > 1) & (amount_children <= 2):
        return 'два ребенка'
    return 'больше двух детей'

<font color='blue'> Категоризировать по количеству детей не надо, это сокращает информативность данных.</font>

Проверим результат:

In [41]:
print(children_group(0))
print(children_group(1))
print(children_group(2))
print(children_group(3))
print(children_group(20))

без детей
один ребенок
два ребенка
больше двух детей
больше двух детей


Добавим столбец с категоризацией в таблицу:

In [42]:
solvency['children_type'] = solvency['children'].apply(children_group)

проверим результат:

In [43]:
solvency.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,purpose_type,children_type
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,"[покупка, , жилье, \n]",недвижимость,один ребенок
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,один ребенок
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,"[покупка, , жилье, \n]",недвижимость,без детей
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,"[дополнительный, , образование, \n]",образование,больше двух детей
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,без детей


Напишем функцию для категоризации по уровню дохода. На основе данных Росстата и аналитических статей в интернете разделим граждан РФ по уровню дохода слудющим образом:
1. малоимущие - меньше 36 000 руб/мес
2. не бедные - от 37 000 руб/мес до 99 000 руб/мес
3. средний класс - от 100 000 руб/мес до 249 000 руб/мес
4. состоятельные - от 250 000 руб/мес до 499 000 руб/мес
5. богатые - от 500 000 руб/мес до 999 999 руб/мес
6. миллионеры - более 1 000 00 руб/мес

<font color='blue'> Возможно, но обычно разделяют по квантилям, полсколькук наша выборка может отличаться от общероссийской.</font>

In [44]:
def income_group(salary):
    if salary <= 36000:
        return 'малоимущие'
    if (salary > 36000) & (salary <= 99000):
        return 'не бедные'
    if (salary > 99000) & (salary <= 249000):
        return 'средний класс'
    if (salary > 250000) & (salary <= 499000):
        return 'состоятельные'
    if (salary > 499000) & (salary <= 999000):
        return 'богатые'
    return 'миллионеры'

Проверим результат:

In [45]:
print(income_group(35000))
print(income_group(99000))
print(income_group(100000))
print(income_group(400000))
print(income_group(800000))
print(income_group(1000000000))

малоимущие
не бедные
средний класс
состоятельные
богатые
миллионеры


Добавим столбец с категоризацией в таблицу:

In [46]:
solvency['income_group'] = solvency['total_income'].apply(income_group)

<font color='blue'> Хорошо, отличная работа.</font>

Проверим результат:

In [47]:
solvency.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,purpose_type,children_type,income_group
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,"[покупка, , жилье, \n]",недвижимость,один ребенок,состоятельные
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,один ребенок,средний класс
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,"[покупка, , жилье, \n]",недвижимость,без детей,средний класс
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,"[дополнительный, , образование, \n]",образование,больше двух детей,состоятельные
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,без детей,средний класс


### Вывод

Теперь заемщики категоризированы по количеству детей и уровню ежемесячного дохода. 

Категоризация по количеству детей поможет при выявлении зависимости между наличием детей и возвратом кредита в срок группой "больше двух детей" (разделение по категориям "0 детей", "1 ребенок", "2 ребенка" мы можем использовать из столбца children), а без категоризации по уровню дохода для выявления зависимости между уровнем дохода и возвратом кредита в срок не обойтись вовсе. Двигаемся дальше.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Перед началом анализа представим данные по соответсвующей категории в сводной таблице:

In [48]:
pivottable_children = solvency[['children_type', 'debt', 'children']].pivot_table(index='children_type', columns='debt', aggfunc='count')

In [49]:
pivottable_children

children      
debt                     0     1
children_type                   
без детей            13027  1063
больше двух детей      417    39
два ребенка           1858   194
один ребенок          4364   444

Также оценим, какова общая доля заемщиков, имевших проблемы с выплатой кредита в срок:

In [50]:
solvency[solvency['debt'] == 1]['debt'].count()

1740

Доля "должников" составляет ~8%

Еще раз посмотрим на категории заемщиков по разрезу "количество детей":

In [51]:
solvency['children_type'].unique()

array(['один ребенок', 'без детей', 'больше двух детей', 'два ребенка'],
      dtype=object)

Если мы узнаем % должников (заемщиков, имевших задолженноть по кредитам) по группам внутри выбранного блока категоризации, мы определим вероятность того, что клиент, принадлежащий к определенной категории, не сможет выплатить кредит в срок.
Рассчитаем вероятности и сравним их.

Опеределим общее количество заемщиков, у которых нет детей:

In [52]:
total_without_children = solvency[solvency['children_type'] == 'без детей']['children_type'].count()

In [53]:
print(total_without_children)

14090


Определим количество заемщиков, у которых нет детей и которые имели задолженность по возврату кредитов:

In [54]:
debtor_without_children_inside = solvency[(solvency['children_type'] == 'без детей') & (solvency['debt'] == 1)]['children_type'].count()

In [55]:
print(debtor_without_children_inside)

1063


Выделим из общего числа заемщиков, не имеющих детей, долю имевших задолженность по возврату кредитов:

In [56]:
percent_debtor_without_children_inside = debtor_without_children_inside / total_without_children

In [57]:
print('Процент должников внутри категории "без детей": {:.2%}'.format(percent_debtor_without_children_inside))

Процент должников внутри категории "без детей": 7.54%


Определим общее количество заемщиков, у которых есть 1 ребенок:

In [58]:
total_one_children = solvency[solvency['children_type'] == 'один ребенок']['children_type'].count()

In [59]:
print(total_one_children)

4808


Определим количество заемщиков, у которых есть 1 ребенок и которые имели задолженность по возврату кредитов:

In [60]:
debtor_one_children_inside = solvency[(solvency['children_type'] == 'один ребенок') & (solvency['debt'] == 1)]['children_type'].count()

In [61]:
print(debtor_one_children_inside)

444


Выделим из общего числа заемщиков, имеющих одного ребенка, долю имевших задолженность по возврату кредитов:

In [62]:
percent_debtor_one_children_inside = debtor_one_children_inside / total_one_children

In [63]:
print('Процент должников внутри категории "один ребенок": {:.2%}'.format(percent_debtor_one_children_inside))

Процент должников внутри категории "один ребенок": 9.23%


Определим общее количество заемщиков, у которых есть 2 ребенка:

In [64]:
total_two_children = solvency[solvency['children_type'] == 'два ребенка']['children_type'].count()

In [65]:
print(total_two_children)

2052


Определим количество заемщиков, у которых есть 2 ребенка и которые имели задолженность по возврату кредитов:

In [66]:
debtor_two_children_inside = solvency[(solvency['children_type'] == 'два ребенка') & (solvency['debt'] == 1)]['children_type'].count()

In [67]:
print(debtor_two_children_inside)

194


Выделим из общего числа заемщиков, имеющих двух детей, долю имевших задолженность по возврату кредитов:

In [68]:
percent_debtor_two_children_inside = debtor_two_children_inside / total_two_children

In [69]:
print('Процент должников внутри категории "два ребенка": {:.2%}'.format(percent_debtor_two_children_inside))

Процент должников внутри категории "два ребенка": 9.45%


Определим общее количество заемщиков, у которых более двух детей:

In [70]:
total_more_than_two_children = solvency[solvency['children_type'] == 'больше двух детей']['children_type'].count()

In [71]:
print(total_more_than_two_children)

456


Определим количество заемщиков, у которых более двух детей и которые имели задолженность по возврату кредитов:

In [72]:
debtor_more_than_two_children_inside = solvency[(solvency['children_type'] == 'больше двух детей') & (solvency['debt'] == 1)]['children_type'].count()

In [73]:
print(debtor_more_than_two_children_inside)

39


Выделим из общего числа заемщиков, имеющих более двух детей, долю имевших задолженность по возврату кредитов:

In [74]:
percent_debtor_more_than_two_children_inside = debtor_more_than_two_children_inside / total_more_than_two_children

In [75]:
print('Процент должников внутри категории "больше двух детей": {:.2%}'.format(percent_debtor_more_than_two_children_inside))

Процент должников внутри категории "больше двух детей": 8.55%


Мы рассчитали % должников по группам внутри блока категоризации "количество детей". Соберем информацию вместе:

In [76]:
print('Процент должников внутри категории "без детей": {:.2%}'.format(percent_debtor_without_children_inside))
print('Процент должников внутри категории "один ребенок": {:.2%}'.format(percent_debtor_one_children_inside))
print('Процент должников внутри категории "два ребенка": {:.2%}'.format(percent_debtor_two_children_inside))
print('Процент должников внутри категории "больше двух детей": {:.2%}'.format(percent_debtor_more_than_two_children_inside))

Процент должников внутри категории "без детей": 7.54%
Процент должников внутри категории "один ребенок": 9.23%
Процент должников внутри категории "два ребенка": 9.45%
Процент должников внутри категории "больше двух детей": 8.55%


### Вывод

- Наиболее расположенной к невыплате кредитов оказалась категория заемщиков с двумя детьми.
- Наименее расположенной к невыплате кредитов оказалась категория заемщиков детей.
- Заемщики без детей в среднем на ~2% чаще погашают кредит без задолженностей, чем заемщики с двумя детьми.
- Заемщики с одним ребенком близки по показателям к заемщикам с двумя детьми - разница составляет ~0.2%.
- Заемщики, имеющие более двух детей, находятся посередине между наиболее и наименее расположенными к невыплате кредитов группами - на ~1% более благонадежны, чем заемщики с двумя детьми и на ~1% менее благонадежны, чем заемщики без детей.

Глобально оценивая зависимость между наличием детей и возвратом кредита в срок, имеем в виду, что общая доля заемщиков с проблемами по выплатам кредита в срок составляет ~8%. Разницу в 2% между максимальным и минимальным значением по вероятности невыплаты кредита в срок в завсимости от количества детей, можем оценить, как значимую. Таким образом, можем заключить, что завимисимость выплаты кредита в срок от наличия детей существует, но не является яркой.

<font color='blue'> Можно было бы прямо в pivot_table посчитать по принципу table[1]/(table[1]+table[0]).</font>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Перед началом анализа представим данные по соответсвующей категории в сводной таблице:

In [77]:
pivottable_family = solvency[['family_status_id', 'debt', 'family_status']].pivot_table(index='family_status', columns='debt', aggfunc='count')

In [78]:
pivottable_family

family_status_id     
debt                                 0    1
family_status                              
Не женат / не замужем             2531  274
в разводе                         1106   85
вдовец / вдова                     892   63
гражданский брак                  3757  388
женат / замужем                  11380  930

Еще раз посмотрим на категории заемщиков по разрезу "семейное положение":

In [79]:
solvency['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [80]:
solvency['family_status_id'].unique()

array([0, 1, 2, 3, 4])

Если мы узнаем % должников (заемщиков, имевших задолженноть по кредитам) по группам внутри выбранного блока категоризации, мы определим вероятность того, что клиент, принадлежащий к определенной категории, не сможет выплатить кредит в срок.

Рассчитаем вероятности и сравним их по аналогии с анализом зависимости между наличием детей и возвратом крдита в срок:

In [81]:
total_married = solvency[solvency['family_status_id'] == 0]['family_status_id'].count()

In [82]:
print(total_married)

12310


In [83]:
debtor_married_inside = solvency[(solvency['family_status_id'] == 0) & (solvency['debt'] == 1)]['family_status_id'].count()

In [84]:
print(debtor_married_inside)

930


In [85]:
percent_debtor_married_inside = debtor_married_inside / total_married

In [86]:
print('Процент должников внутри категории "женат / замужем": {:.2%}'.format(percent_debtor_married_inside))

Процент должников внутри категории "женат / замужем": 7.55%


In [87]:
total_civil_marriage = solvency[solvency['family_status_id'] == 1]['family_status_id'].count()

In [88]:
print(total_civil_marriage)

4145


In [89]:
debtor_civil_marriage_inside = solvency[(solvency['family_status_id'] == 1) & (solvency['debt'] == 1)]['family_status_id'].count()

In [90]:
print(debtor_civil_marriage_inside)

388


In [91]:
percent_debtor_civil_marriage_inside = debtor_civil_marriage_inside / total_civil_marriage

In [92]:
print('Процент должников внутри категории "гражданский брак": {:.2%}'.format(percent_debtor_civil_marriage_inside))

Процент должников внутри категории "гражданский брак": 9.36%


In [93]:
total_widow = solvency[solvency['family_status_id'] == 2]['family_status_id'].count()

In [94]:
print(total_widow)

955


In [95]:
debtor_widow_inside = solvency[(solvency['family_status_id'] == 2) & (solvency['debt'] == 1)]['family_status_id'].count()

In [96]:
print(debtor_widow_inside)

63


In [97]:
percent_debtor_widow_inside = debtor_widow_inside / total_widow

In [98]:
print('Процент должников внутри категории "вдовец / вдова": {:.2%}'.format(percent_debtor_widow_inside))

Процент должников внутри категории "вдовец / вдова": 6.60%


In [99]:
total_divorce = solvency[solvency['family_status_id'] == 3]['family_status_id'].count()

In [100]:
print(total_divorce)

1191


In [101]:
debtor_divorce_inside = solvency[(solvency['family_status_id'] == 3) & (solvency['debt'] == 1)]['family_status_id'].count()

In [102]:
print(debtor_divorce_inside)

85


In [103]:
percent_debtor_divorce_inside = debtor_divorce_inside / total_divorce

In [104]:
print('Процент должников внутри категории "в разводе": {:.2%}'.format(percent_debtor_divorce_inside))

Процент должников внутри категории "в разводе": 7.14%


In [105]:
total_unmarried = solvency[solvency['family_status_id'] == 4]['family_status_id'].count()

In [106]:
print(total_unmarried)

2805


In [107]:
debtor_unmarried_inside = solvency[(solvency['family_status_id'] == 4) & (solvency['debt'] == 1)]['family_status_id'].count()

In [108]:
print(debtor_unmarried_inside)

274


In [109]:
percent_debtor_unmarried_inside = debtor_unmarried_inside / total_unmarried

In [110]:
print('Процент должников внутри категории "не женат / не замужем": {:.2%}'.format(percent_debtor_unmarried_inside))

Процент должников внутри категории "не женат / не замужем": 9.77%


Мы рассчитали % должников по группам внутри блока категоризации "семейное положение". Соберем информацию вместе:

In [111]:
print('Процент должников внутри категории "женат / замужем": {:.2%}'.format(percent_debtor_married_inside))
print('Процент должников внутри категории "гражданский брак": {:.2%}'.format(percent_debtor_civil_marriage_inside))
print('Процент должников внутри категории "вдовец / вдова": {:.2%}'.format(percent_debtor_widow_inside))
print('Процент должников внутри категории "в разводе": {:.2%}'.format(percent_debtor_divorce_inside))
print('Процент должников внутри категории "не женат / не замужем": {:.2%}'.format(percent_debtor_unmarried_inside))

Процент должников внутри категории "женат / замужем": 7.55%
Процент должников внутри категории "гражданский брак": 9.36%
Процент должников внутри категории "вдовец / вдова": 6.60%
Процент должников внутри категории "в разводе": 7.14%
Процент должников внутри категории "не женат / не замужем": 9.77%


### Вывод

- Наиболее расположенной к невыплате кредитов оказалась категория не женатых/не замужних заемщиков.
- Наименее расположенной к невыплате кредитов оказалась категория заемщиков-вдов/вдовцов.
- Вдовцы/вдовы в среднем на ~3% чаще погашают кредит без задолженностей, чем не женатые/не замужние заемщики.
- Заемщики в гражданском браке по вероятности не выплаты кредита в срок стремятся к не женатым/не замужним заемщикам - разница составляет ~0.6%.
- Заемщики в разводе по вероятности не выплаты кредита в срок стремятся к вдовам/вдовцам - разница составляет ~0.5%.
- Замужние/женатые заемщикипо вероятности не выплаты кредита в срок стремятся к вдовам/вдовцам - разница составляет менее 1%.

Глобально оценивая зависимость между семейным положением и возвратом кредита в срок, имеем в виду, что общая доля заемщиков с проблемами по выплатам кредита в срок составляет ~8%. Разницу в 3% между максимальным и минимальным значением по вероятности невыплаты кредита в срок в завсимости от семейного положения, можем оценить, как значимую. Таким образом, можем заключить, что завимисимость выплаты кредита в срок от семейного положения существует, но не является яркой.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Перед началом анализа представим данные по соответсвующей категории в сводной таблице:

In [112]:
pivottable_income = solvency[['income_group', 'debt', 'total_income']].pivot_table(index='income_group', columns='debt', aggfunc='count')

In [113]:
pivottable_income

total_income      
debt                     0     1
income_group                    
богатые                189    12
малоимущие              71     4
миллионеры              57     2
не бедные             3925   342
состоятельные         2402   180
средний класс        13022  1200

Еще раз посмотрим на категории заемщиков по разрезу "уровень дохода":

In [114]:
solvency['income_group'].unique()

array(['состоятельные', 'средний класс', 'не бедные', 'богатые',
       'малоимущие', 'миллионеры'], dtype=object)

Если мы узнаем % должников (заемщиков, имевших задолженноть по кредитам) по группам внутри выбранного блока категоризации, мы определим вероятность того, что клиент, принадлежащий к определенной категории, не сможет выплатить кредит в срок.

Рассчитаем вероятности и сравним их по аналогии с анализом зависимости между наличием детей и возвратом крдита в срок:

In [115]:
total_penniless = solvency[solvency['income_group'] == 'малоимущие']['income_group'].count()

In [116]:
print(total_penniless)

75


In [117]:
debtor_penniless_inside = solvency[(solvency['income_group'] == 'малоимущие') & (solvency['debt'] == 1)]['income_group'].count()

In [118]:
print(debtor_penniless_inside)

4


In [119]:
percent_debtor_penniless_inside = debtor_penniless_inside / total_penniless

In [120]:
print('Процент должников внутри категории "малоимущие": {:.2%}'.format(percent_debtor_penniless_inside))

Процент должников внутри категории "малоимущие": 5.33%


In [121]:
total_notpenniless = solvency[solvency['income_group'] == 'не бедные']['income_group'].count()

In [122]:
print(total_notpenniless)

4267


In [123]:
debtor_notpenniless_inside = solvency[(solvency['income_group'] == 'не бедные') & (solvency['debt'] == 1)]['income_group'].count()

In [124]:
print(debtor_notpenniless_inside)

342


In [125]:
percent_debtor_notpenniless_inside = debtor_notpenniless_inside / total_notpenniless

In [126]:
print('Процент должников внутри категории "не бедные": {:.2%}'.format(percent_debtor_notpenniless_inside))

Процент должников внутри категории "не бедные": 8.01%


In [127]:
total_middleclass = solvency[solvency['income_group'] == 'средний класс']['income_group'].count()

In [128]:
print(total_middleclass)

14222


In [129]:
debtor_middleclass_inside = solvency[(solvency['income_group'] == 'средний класс') & (solvency['debt'] == 1)]['income_group'].count()

In [130]:
print(debtor_middleclass_inside)

1200


In [131]:
percent_debtor_middleclass_inside = debtor_middleclass_inside / total_middleclass

In [132]:
print('Процент должников внутри категории "средний класс": {:.2%}'.format(percent_debtor_middleclass_inside))

Процент должников внутри категории "средний класс": 8.44%


In [133]:
total_wealthy = solvency[solvency['income_group'] == 'состоятельные']['income_group'].count()

In [134]:
print(total_wealthy)

2582


In [135]:
debtor_wealthy_inside = solvency[(solvency['income_group'] == 'состоятельные') & (solvency['debt'] == 1)]['income_group'].count()

In [136]:
print(debtor_wealthy_inside)

180


In [137]:
percent_debtor_wealthy_inside = debtor_wealthy_inside / total_wealthy

In [138]:
print('Процент должников внутри категории "состоятельные": {:.2%}'.format(percent_debtor_wealthy_inside))

Процент должников внутри категории "состоятельные": 6.97%


In [139]:
total_rich = solvency[solvency['income_group'] == 'богатые']['income_group'].count()

In [140]:
print(total_rich)

201


In [141]:
debtor_rich_inside = solvency[(solvency['income_group'] == 'богатые') & (solvency['debt'] == 1)]['income_group'].count()

In [142]:
print(debtor_rich_inside)

12


In [143]:
percent_debtor_rich_inside = debtor_rich_inside / total_rich

In [144]:
print('Процент должников внутри категории "богатые": {:.2%}'.format(percent_debtor_rich_inside))

Процент должников внутри категории "богатые": 5.97%


In [145]:
total_millionaire = solvency[solvency['income_group'] == 'миллионеры']['income_group'].count()

In [146]:
print(total_millionaire)

59


In [147]:
debtor_millionaire_inside = solvency[(solvency['income_group'] == 'миллионеры') & (solvency['debt'] == 1)]['income_group'].count()

In [148]:
print(debtor_millionaire_inside)

2


In [149]:
percent_debtor_millionaire_inside = debtor_millionaire_inside / total_millionaire

In [150]:
print('Процент должников внутри категории "миллионеры": {:.2%}'.format(percent_debtor_millionaire_inside))

Процент должников внутри категории "миллионеры": 3.39%


Мы рассчитали % должников по группам внутри блока категоризации "уровень дохода". Соберем информацию вместе:

In [151]:
print('Процент должников внутри категории "малоимущие": {:.2%}'.format(percent_debtor_penniless_inside))
print('Процент должников внутри категории "не бедные": {:.2%}'.format(percent_debtor_notpenniless_inside))
print('Процент должников внутри категории "средний класс": {:.2%}'.format(percent_debtor_middleclass_inside))
print('Процент должников внутри категории "состоятельные": {:.2%}'.format(percent_debtor_wealthy_inside))
print('Процент должников внутри категории "богатые": {:.2%}'.format(percent_debtor_rich_inside))
print('Процент должников внутри категории "миллионеры": {:.2%}'.format(percent_debtor_millionaire_inside))

Процент должников внутри категории "малоимущие": 5.33%
Процент должников внутри категории "не бедные": 8.01%
Процент должников внутри категории "средний класс": 8.44%
Процент должников внутри категории "состоятельные": 6.97%
Процент должников внутри категории "богатые": 5.97%
Процент должников внутри категории "миллионеры": 3.39%


### Вывод

- Наиболее расположенной к невыплате кредитов оказалась категория заемщиков из среднего класса.
- Наименее расположенной к невыплате кредитов оказалась категория миллионеров.
- Миллионеры в среднем на ~5% чаще погашают кредит без задолженностей, чем представители среднего класса.
- Заемщики в категории "не бедные" по вероятности не выплаты кредита в срок стремятся к представителям среднего класса - разница составляет менее половины процента.
- Не учитывая категорию миллионеров,наименее расположенной к невыплате кредитов оказалась категория малоимущих - вероятность выплатить кредит у данной категории на ~3% выше, чем у представителей среднего класса.
- К категории малоимущих по вероятности выплатить кредит в срок стремится категория "богатые" - разница составляет ~0.6%.
- Категория "состоятельные" по вероятности невыплаты кредита в срок находится ближе к наименее благонадежному среднему классу, чем к наиболее благонадежным миллионерам и малоимущим.

Глобально оценивая зависимость между уровнем дохода и возвратом кредита в срок, имеем в виду, что общая доля заемщиков с проблемами по выплатам кредита в срок составляет ~8%. Разницу в 5% между максимальным и минимальным значением по вероятности невыплаты кредита в срок в завсимости от уровня дохода, можем оценить, как значимую. Таким образом, можем заключить, что завимисимость выплаты кредита в срок зависит от уровня дохода.

- Как разные цели кредита влияют на его возврат в срок?

Перед началом анализа представим данные по соответсвующей категории в сводной таблице:

In [152]:
pivottable_purpose = solvency[['purpose_type', 'debt', 'purpose']].pivot_table(index='purpose_type', columns='debt', aggfunc='count')

In [153]:
pivottable_purpose

purpose     
debt               0    1
purpose_type             
автомобиль      3893  402
недвижимость   10004  782
образование     3633  370
свадьба         2136  186

Посмотрим на уникальные значения столбца purpose_type:

In [154]:
solvency['purpose_type'].unique()

array(['недвижимость', 'автомобиль', 'образование', 'свадьба'],
      dtype=object)

Если мы узнаем % должников (заемщиков, имевших задолженноть по кредитам) по группам внутри выбранного блока целей кредита, мы определим вероятность того, что клиент, принадлежащий к определенной категории, не сможет выплатить кредит в срок.

Рассчитаем вероятности и сравним их по аналогии с анализом зависимости между наличием детей и возвратом крдита в срок:

In [155]:
total_realty = solvency[solvency['purpose_type'] == 'недвижимость']['purpose_type'].count()

In [156]:
print(total_realty)

10786


In [157]:
debtor_realty_inside = solvency[(solvency['purpose_type'] == 'недвижимость') & (solvency['debt'] == 1)]['purpose_type'].count()

In [158]:
print(debtor_realty_inside)

782


In [159]:
percent_debtor_realty_inside = debtor_realty_inside / total_realty

In [160]:
print('Процент должников внутри категории "недвижимость": {:.2%}'.format(percent_debtor_realty_inside))

Процент должников внутри категории "недвижимость": 7.25%


In [161]:
total_auto = solvency[solvency['purpose_type'] == 'автомобиль']['purpose_type'].count()

In [162]:
print(total_auto)

4295


In [163]:
debtor_auto_inside = solvency[(solvency['purpose_type'] == 'автомобиль') & (solvency['debt'] == 1)]['purpose_type'].count()

In [164]:
print(debtor_auto_inside)

402


In [165]:
percent_debtor_auto_inside = debtor_auto_inside / total_auto

In [166]:
print('Процент должников внутри категории "автомобиль": {:.2%}'.format(percent_debtor_auto_inside))

Процент должников внутри категории "автомобиль": 9.36%


In [167]:
total_education = solvency[solvency['purpose_type'] == 'образование']['purpose_type'].count()

In [168]:
print(total_education)

4003


In [169]:
debtor_education_inside = solvency[(solvency['purpose_type'] == 'образование') & (solvency['debt'] == 1)]['purpose_type'].count()

In [170]:
print(debtor_education_inside)

370


In [171]:
percent_debtor_education_inside = debtor_education_inside / total_education

In [172]:
print('Процент должников внутри категории "образование": {:.2%}'.format(percent_debtor_education_inside))

Процент должников внутри категории "образование": 9.24%


In [173]:
total_wedding = solvency[solvency['purpose_type'] == 'свадьба']['purpose_type'].count()

In [174]:
print(total_wedding)

2322


In [175]:
debtor_wedding_inside = solvency[(solvency['purpose_type'] == 'свадьба') & (solvency['debt'] == 1)]['purpose_type'].count()

In [176]:
print(debtor_wedding_inside)

186


In [177]:
percent_debtor_wedding_inside = debtor_wedding_inside / total_wedding

In [178]:
print('Процент должников внутри категории "свадьба": {:.2%}'.format(percent_debtor_wedding_inside))

Процент должников внутри категории "свадьба": 8.01%


Мы рассчитали % должников по группам внутри блока категоризации "цель кредита". Соберем информацию вместе:

In [179]:
print('Процент должников внутри категории "недвижимость": {:.2%}'.format(percent_debtor_realty_inside))
print('Процент должников внутри категории "автомобиль": {:.2%}'.format(percent_debtor_auto_inside))
print('Процент должников внутри категории "образование": {:.2%}'.format(percent_debtor_education_inside))
print('Процент должников внутри категории "свадьба": {:.2%}'.format(percent_debtor_wedding_inside))

Процент должников внутри категории "недвижимость": 7.25%
Процент должников внутри категории "автомобиль": 9.36%
Процент должников внутри категории "образование": 9.24%
Процент должников внутри категории "свадьба": 8.01%


### Вывод

- Наиболее расположенной к невыплате кредитов оказалась категория заемщиков с целью кредита "автомобиль".
- Наименее расположенной к невыплате кредитов оказалась категория заемщиков с целью кредита "недвижимость".
- Заемщики, берущие кредит на недвижимость в среднем на ~2% процента чаще выплачивают кредит без в срок, чем заемщики, берущие кредит на автомобиль.
-  Заемщики, берущие кредит на образование по вероятности не выплаты кредита в срок стремятся к заемщикам, берущим кредит на автомобиль - разница составляет ~0.1%
-  Заемщики, берущие кредит на свадьбу по вероятности не выплаты кредита в срок стремятся к заемщикам, берущим кредит на недвижимость - разница составляет ~0.8%

Глобально оценивая зависимость между уровнем дохода и возвратом кредита в срок, имеем в виду, что общая доля заемщиков с проблемами по выплатам кредита в срок составляет ~8%. Разницу в 2% между максимальным и минимальным значением по вероятности невыплаты кредита в срок в завсимости от уровня дохода, можем оценить, как значимую. Таким образом, можем заключить, что завимисимость выплаты кредита в срок от цели кредита существует, но является не яркой.

<font color='blue'> Ответы на вопросы выполнены верно.</font>

### Шаг 4. Общий вывод

Мы выявили,что семейное положение и количество детей, а также уровень дохода и цель кредита могут влиять на возврат кредита в срок, хоть и не увидели ярких зависимостей. Можно предположить, что на возврат кредита в срок влияют сразу много факторов одновременно, в том числе могут воздействовать различные непредугадываемые обстоятельства. 

Наиболее прослеживаемую зависимость среди обозначенных гипотез мы увидели между возвратом кредита в скрок и уровнем дохода. Данный факт можно объяснить, так как кредитные обязательства это прежде всего финансовые отношения между кредитором и заемщиком, а уровень ежемесячного дохода заемщика в сильной мере должен определять возможность выполнять финансовые обязательства. Отсутствие прямой зависимости "выше доход - ниже вероятность выплаты кредита не в срок" также можно объяснить наличием дополнительных факторов, действующих одновременно. 

В общем результате гипотезы существующего влияния проанализированных факторов на возврат кредита в срок подтверждены.

<font color='blue'> Проект выполнен отлично, молодец. Классный старт, так держать. Удачи в следующих проектах.</font>

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.